In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from procurement_lib import GoogleSheet

In [53]:
# df_gtin = GoogleSheet("1TBAwmxAoqg6FX79d7wLu3DVtIiOb6gRfkN5fMA_tLNc")
# df_gtin = df_gtin.get_as_dataframe('Hoja 1')

In [54]:
# query = """
# select
#     competitor.competitor_name,
#     site.identifier_value as site_code,
#     quotation_date.full_date AS quotation_date,
#     su.source_id,
#     ROUND(AVG(cpp.product_selected_price),2)::float as price
# from dpr_product_pricing.fact_collected_product_prices cpp
#     inner join dpr_shared.dim_date quotation_date
#         on cpp.dim_quotation_date = quotation_date.date_id
#     inner join dpr_shared.dim_time quotation_time
#         on cpp.dim_quotation_time = quotation_time.time_id
#     inner join dpr_shared.dim_site site
#         on cpp.dim_site = site.site_id
#     inner join dpr_shared.dim_category cat
#         on cpp.dim_category = cat.category_id
#     inner join dpr_product_pricing.dim_product_outlier_type outlier_type
#         on cpp.dim_outlier_type = outlier_type.outlier_type_id
#     inner join dpr_product_pricing.dim_product_source_type source_type
#         on cpp.dim_source_type = source_type.source_type_id
#     inner join dpr_product_pricing.dim_product_competitor competitor
#         on cpp.dim_competitor = competitor.competitor_id
#     inner join dpr_product_pricing.dim_product_competitor_type competitor_type
#         on(
#             case
#                 when cpp.super_category = 'Fruver'
#                     then competitor.product_competitor_type_id_fruver = competitor_type.competitor_type_id
#                 when cpp.super_category = 'Multicategoría'
#                     then competitor.product_competitor_type_id_multicategoria = competitor_type.competitor_type_id
#             end
#         )
#     inner join dpr_shared.dim_stock_unit su
#         on cpp.dim_stock_unit = su.stock_unit_id
# where quotation_date.full_date >= '2024-02-10'
#     AND quotation_date.full_date < '2024-06-01'
#     AND competitor.competitor_name NOT ILIKE '%cayena%'
#     AND su.source_id IN {skus}
#     AND site.identifier_value IN ('SPO','CWB','VCP','BHZ')
# GROUP BY 1,2,3,4
# """.format(skus=tuple(df_gtin.source_id.unique()))
# df_zkkkkk = run_read_dwd_query(query)

In [55]:
# df_zkkkkk = pd.merge(df_zkkkkk, df_gtin, left_on=['source_id','site_code'], right_on=['source_id','city'], how='left')
# df_zkkkkk = df_zkkkkk.drop(['city', 'GTIN'], axis=1)
# df_zkkkkk = df_zkkkkk.dropna().reset_index(drop=True)

# df_zkkkkk['lifetime'] = 8

In [56]:
# # Ensure dataframe is sorted by 'quotation_date'
# df_zkkkkk = df_zkkkkk.sort_values(by='quotation_date')

# # Generate the required rows for missing dates
# new_rows = []

# for (competitor, source_id), group in df_zkkkkk.groupby(['competitor_name', 'source_id']):
#     group = group.sort_values(by='quotation_date')
#     last_known_price = None
#     last_known_date = None
#     lifetime = 8
    
#     for current_index in range(len(group)):
#         current_date = group.iloc[current_index]['quotation_date']
#         price = group.iloc[current_index]['price']
        
#         # If this is not the first iteration, fill in missing dates
#         if last_known_date is not None:
#             days_diff = (current_date - last_known_date).days
#             if days_diff > 1:
#                 for j in range(1, min(days_diff, lifetime + 1)):
#                     new_date = last_known_date + timedelta(days=j)
#                     new_row = {
#                         'site_code': group.iloc[current_index]['site_code'],
#                         'quotation_date': new_date,
#                         'competitor_name': competitor,
#                         'source_id': source_id,
#                         'product_name': group.iloc[current_index]['product_name'],
#                         'price': last_known_price,
#                         'lifetime': lifetime - j
#                     }
#                     new_rows.append(new_row)
                    
#                     # Stop if we reach a new datapoint date
#                     if new_date + timedelta(days=1) == current_date:
#                         break
        
#         # Update the last known values and reset lifetime
#         last_known_price = price
#         last_known_date = current_date
#         lifetime = 8  # Reset lifetime

#     # After processing all known dates for the group, continue generating rows until lifetime reaches 0
#     while lifetime > 0:
#         last_known_date += timedelta(days=1)
#         new_row = {
#             'site_code': group.iloc[-1]['site_code'],
#             'quotation_date': last_known_date,
#             'competitor_name': competitor,
#             'source_id': source_id,
#             'product_name': group.iloc[-1]['product_name'],
#             'price': last_known_price,
#             'lifetime': lifetime - 1
#         }
#         new_rows.append(new_row)
#         lifetime -= 1

# # Append new rows to the dataframe
# df_zkkkkk = df_zkkkkk.append(new_rows, ignore_index=True)

# # Sort the final dataframe
# df_zkkkkk = df_zkkkkk.sort_values(by=['competitor_name', 'source_id', 'quotation_date'])
# df_zkkkkk['replica'] = df_zkkkkk['lifetime'] == 8

In [57]:
# df_zkkkkk.to_csv("info_zk_estadistic.csv",index=False)

In [58]:
# summary_df = df_zkkkkk.groupby(['quotation_date', 'source_id','product_name'])['price'].describe(percentiles=[0.10, 0.25, 0.5, 0.75]).reset_index()
# summary_df = summary_df[['quotation_date', 'source_id','product_name', 'min', '10%', '25%', '50%', '75%', 'max']]
# summary_df.columns = ['quotation_date', 'source_id','product_name', 'min', 'p10','q1', 'median', 'q3', 'max'] #ESTABA CON 3T

In [59]:
# summary_df.to_csv("grafica_B_zk.csv",index=False)

In [2]:
df_A = pd.read_csv("grafica_A_ip.csv")

df_A['quotation_date'] = pd.to_datetime(df_A['quotation_date'])

df_B = pd.read_csv("grafica_B_zk.csv")

df_B['quotation_date'] = pd.to_datetime(df_B['quotation_date'])

In [3]:
merged_df = pd.merge(df_A, df_B[['quotation_date','source_id','min', 'p10', 'q1',
       'median', 'q3', 'max']], left_on=['quotation_date', 'source_id'], right_on=['quotation_date', 'source_id'], how='left')

In [4]:
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact

# Function to plot data for a single product with additional metrics
def plot_product_prices_with_complement(product_name):
    product_data = merged_df[merged_df['product_name'] == product_name]

    fig = go.Figure()

    # Plot the summary statistics
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['min_x'], mode='lines', name='Min_IP'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['q1_x'], mode='lines', name='Q1_IP'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['p10_x'], mode='lines', name='P10_IP'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['median_x'], mode='lines', name='Median_IP'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['q3_x'], mode='lines', name='Q3_IP'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['max_x'], mode='lines', name='Max_IP'))

    # Plot the summary statistics
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['min_y'], mode='lines', line=dict(dash='dash'),  name='Min_ZK'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['q1_y'], mode='lines', line=dict(dash='dash'), name='Q1_ZK'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['p10_y'], mode='lines', line=dict(dash='dash'), name='P10_ZK'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['median_y'], mode='lines', line=dict(dash='dash'), name='Median_ZK'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['q3_y'], mode='lines', line=dict(dash='dash'), name='Q3_ZK'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['max_y'], mode='lines', line=dict(dash='dash'), name='Max_ZK'))
        
    fig.update_layout(
        title=f'Price Summary and Additional Metrics for Product ID: {product_name}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Metrics',
        height=800,
        template='plotly_white'
    )

    fig.show()

# Get unique product IDs
product_names = pd.Series(merged_df.product_name.unique()).sort_values()

# Create dropdown widget
dropdown = widgets.Dropdown(options=product_names, description='Product Name:')

# Interact function to update plot based on dropdown selection
interact(plot_product_prices_with_complement, product_name=dropdown)

interactive(children=(Dropdown(description='Product Name:', options=('BHZ || Achocolatado em Pó Nescau 2.0 Sac…

<function __main__.plot_product_prices_with_complement(product_name)>